In [1]:
import pandas as pd

# Read the Excel file

file_path = r"D:\Datascience\traffic_stops_cleaned (1).xlsx"
df = pd.read_excel(file_path)
df.head()
df = df.bfill().ffill()
df
df.to_excel("cleaned_data.xlsx", index=False)
df.to_excel(r"C:\Users\IBuild\Downloads\cleaned_data.xlsx", index=False)
data_list = df.values.tolist()

# connecting to SQL
import mysql.connector
conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Saiswarna@28599"
)
cursor_mysql=conn_mysql.cursor()
print("Mysql connection established successfully")

#Creating a data base in Mysql
cursor_mysql.execute("CREATE DATABASE Police_10_db")
print("Database created successfully")

#Creating a table in mysql
cursor_mysql.execute("USE Police_10_db")

cursor_mysql.execute("""
CREATE TABLE Police_10(
    Stop_date DATE,
    Stop_Time TIME,
    Country_Name VARCHAR(40),
    Driver_Gender VARCHAR(10),
    Driver_Raw_Age INT,
    Driver_Age INT,
    Driver_Race VARCHAR(20),
    Violation_Raw VARCHAR(70),
    Violation VARCHAR(40),
    Search_Conducted VARCHAR(50),
    Search_Type VARCHAR(30),
    Stop_Outcome VARCHAR(30),
    Is_Arrested VARCHAR(30),
    Stop_Duration VARCHAR(20), 
    Drug_Related_Stop VARCHAR(10),
    Vehicle_Number VARCHAR(30)
);
""")

conn_mysql.commit()
print("Table has created successfully in mysql")

#Inserting values into table

data_list = df.values.tolist()

query = """
INSERT INTO Police_10 (
    Stop_date, Stop_Time, Country_Name, Driver_Gender, Driver_Raw_Age, Driver_Age,
    Driver_Race, Violation_Raw, Violation, Search_Conducted, Search_Type, Stop_Outcome,
    Is_Arrested, Stop_Duration, Drug_Related_Stop, Vehicle_Number
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")
conn_mysql.commit()

print("Data inserted Successfully! (No NaN values)")

#SQL Queries

# Top 10 vehicle numbers involved in drug-related stops

query = """
SELECT Vehicle_Number, COUNT(*) AS Drug_Stop_Count
FROM Police_10
WHERE Drug_Related_Stop = TRUE
GROUP BY Vehicle_Number
ORDER BY Drug_Stop_Count DESC
LIMIT 10;
"""

# Which vehicles were most frequently searched?

query = """
SELECT Vehicle_Number, COUNT(*) AS Search_Count
FROM Police_10
WHERE Search_Conducted = TRUE
GROUP BY Vehicle_Number
ORDER BY Search_Count DESC;
"""

#Which driver age group had the highest arrest rate?

query = """
SELECT 
    CASE 
        WHEN Driver_Age < 18 THEN 'Under 18'
        WHEN Driver_Age BETWEEN 18 AND 25 THEN '18-25'
        WHEN Driver_Age BETWEEN 26 AND 40 THEN '26-40'
        WHEN Driver_Age BETWEEN 41 AND 60 THEN '41-60'
        ELSE '60+'
    END AS Age_Group,
    ROUND(AVG(Is_Arrested = TRUE) * 100, 2) AS Arrest_Rate_Percent
FROM Police_10
GROUP BY Age_Group
ORDER BY Arrest_Rate_Percent DESC;
"""

#What is the gender distribution of drivers stopped in each country?

query = """
SELECT 
    Country_Name,
    Driver_Gender,
    COUNT(*) AS Total_Stops
FROM Police_10
GROUP BY Country_Name, Driver_Gender
ORDER BY Country_Name, Total_Stops DESC;
"""

#Which race and gender combination has the highest search rate?

query= """
SELECT 
    Driver_Race,
    Driver_Gender,
    COUNT(*) AS Total_Stops,
    SUM(Search_Conducted = 'Yes') AS Searches_Conducted,
    ROUND((SUM(Search_Conducted = TRUE) / COUNT(*)) * 100, 2) AS Search_Rate_Percent
FROM Police_10
GROUP BY Driver_Race, Driver_Gender
ORDER BY Search_Rate_Percent DESC;
"""

#What time of day sees the most traffic stops?

query="""
SELECT 
    HOUR(Stop_Time) AS Hour_Of_Day,
    COUNT(*) AS Total_Stops
FROM Police_10
GROUP BY Hour_Of_Day
ORDER BY Total_Stops DESC;
"""

#What is the average stop duration for different violations?

query="""
SELECT 
    Violation,
    ROUND(AVG(Stop_Duration), 2) AS Avg_Stop_Duration
FROM Police_10
GROUP BY Violation
ORDER BY Avg_Stop_Duration DESC;
"""

#Are stops during the night more likely to lead to arrests?
query="""
SELECT 
    CASE 
        WHEN Stop_Time >= '20:00:00' OR Stop_Time < '06:00:00' THEN 'Night'
        ELSE 'Day'
    END AS Time_of_Day,
    COUNT(*) AS Total_Stops,
    SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) AS Total_Arrests,
    ROUND(100.0 * SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) AS Arrest_Rate_Percent
FROM Police_10
GROUP BY Time_of_Day;
"""

#Which violations are most associated with searches or arrests?

query="""
SELECT Violation, COUNT(*) AS Arrest_Count
FROM Police_10
WHERE Is_Arrested = TRUE
GROUP BY Violation
ORDER BY Arrest_Count DESC
LIMIT 10;
"""
query="""
SELECT Violation, COUNT(*) AS Search_Count
FROM Police_10
WHERE Search_Conducted = TRUE
GROUP BY Violation
ORDER BY Search_Count DESC
LIMIT 10;
"""
#Which violations are most common among younger drivers (<25)?

query="""
SELECT Violation, COUNT(*) AS Violation_Count
FROM Police_Secure_Check
WHERE Driver_Age < 25
GROUP BY Violation
ORDER BY Violation_Count DESC
LIMIT 10;
"""
#Is there a violation that rarely results in search or arrest?

query="""
SELECT Violation,
       COUNT(*) AS Total_Stops,
       SUM(CASE WHEN Is_Arrested= TRUE THEN 1 ELSE 0 END) AS Arrests,
       ROUND(SUM(CASE WHEN Is_Arrested= TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS Arrest_Percentage
FROM Police_10
GROUP BY Violation
ORDER BY Arrest_Percentage ASC
LIMIT 10;
"""

#Which countries report the highest rate of drug-related stops?

query="""
SELECT Country_Name, 
       COUNT(*) AS Total_Stops,
       SUM(CASE WHEN Drug_Related_Stop = 'TRUE' THEN 1 ELSE 0 END) AS Drug_Related_Stops,
       ROUND(SUM(CASE WHEN Drug_Related_Stop = 'TRUE' THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS Drug_Related_Percentage
FROM Police_10
GROUP BY Country_Name
ORDER BY Drug_Related_Percentage DESC
LIMIT 10;
"""

#What is the arrest rate by country and violation?

query="""
SELECT 
    Country_Name,
    Violation,
    COUNT(*) AS Total_Stops,
    SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) AS Arrests,
    ROUND(SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS Arrest_Rate_Percentage
FROM Police_10
GROUP BY Country_Name, Violation
ORDER BY Arrest_Rate_Percentage DESC, Total_Stops DESC;
"""

#Which country has the most stops with search conducted?

query="""
SELECT 
    Country_Name,
    COUNT(*) AS Search_Stops
FROM Police_10
WHERE Search_Conducted = TRUE
GROUP BY Country_Name
ORDER BY Search_Stops DESC
LIMIT 40;
"""

#Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)

query="""
SELECT 
    Country_Name,
    YEAR(Stop_date) AS Stop_Year,
    COUNT(*) AS Total_Stops,
    SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) AS Total_Arrests,
    ROUND(SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS Arrest_Rate_Percentage,
    RANK() OVER (PARTITION BY YEAR(Stop_date) ORDER BY COUNT(*) DESC) AS Country_Rank_By_Stops
FROM Police_10
GROUP BY Country_Name, Stop_Year
ORDER BY Stop_Year, Total_Stops DESC;
SELECT 
    t.Country_Name,
    t.Stop_Year,
    t.Total_Stops,
    t.Total_Arrests,
    ROUND(t.Total_Arrests / t.Total_Stops * 100, 2) AS Arrest_Rate_Percentage
FROM (
    SELECT 
        Country_Name,
        YEAR(Stop_date) AS Stop_Year,
        COUNT(*) AS Total_Stops,
        SUM(CASE WHEN Is_Arrested= TRUE THEN 1 ELSE 0 END) AS Total_Arrests
    FROM Police_10
    GROUP BY Country_Name, Stop_Year
) AS t
ORDER BY t.Stop_Year, t.Total_Stops DESC;
"""

#Driver Violation Trends Based on Age and Race (Join with Subquery)

query="""
SELECT 
    t.Driver_Race,
    v.Age_Group,
    v.Violation,
    v.Violation_Count
FROM 
    Police_10 AS t
JOIN
    (
        -- Subquery: count violations by age group
        SELECT
            CASE
                WHEN Driver_Age < 25 THEN '<25'
                WHEN Driver_Age BETWEEN 25 AND 34 THEN '25-34'
                WHEN Driver_Age BETWEEN 35 AND 44 THEN '35-44'
                WHEN Driver_Age BETWEEN 45 AND 54 THEN '45-54'
                ELSE '55+'
            END AS Age_Group,
            Violation,
            Driver_Race,
            COUNT(*) AS Violation_Count
        FROM Police_10
        GROUP BY Age_Group, Violation, Driver_Race
    ) AS v
ON t.Driver_Race = v.Driver_Race
ORDER BY t.Driver_Race, v.Age_Group, v.Violation_Count DESC;
"""

#Time Period Analysis of Stops

query="""
SELECT
    t.Year,
    t.Month,
    t.Hour,
    COUNT(*) AS Number_of_Stops
FROM
    Police_10 AS p
JOIN
    (
        -- Subquery to extract year, month, hour
        SELECT
            Stop_date,
            YEAR(Stop_date) AS Year,
            MONTH(Stop_date) AS Month,
            HOUR(Stop_Time) AS Hour
        FROM Police_10
    ) AS t
ON p.Stop_date = t.Stop_date
GROUP BY t.Year, t.Month, t.Hour
ORDER BY t.Year, t.Month, t.Hour;
SELECT
    YEAR(Stop_date) AS Year,
    MONTH(Stop_date) AS Month,
    HOUR(Stop_date) AS Hour,
    COUNT(*) AS Number_of_Stops
FROM Police_10
GROUP BY Year, Month, Hour
ORDER BY Year, Month, Hour;
"""

#Violations with High Search and Arrest Rates (Window Function)

query="""
SELECT
    Violation,
    Total_Stops,
    Searches,
    Arrests,
    ROUND(Searches / Total_Stops * 100, 2) AS Search_Rate_Percent,
    ROUND(Arrests / Total_Stops * 100, 2) AS Arrest_Rate_Percent,
    RANK() OVER (ORDER BY Searches / Total_Stops DESC) AS Search_Rank,
    RANK() OVER (ORDER BY Arrests / Total_Stops DESC) AS Arrest_Rank
FROM
    (
        SELECT
            Violation,
            COUNT(*) AS Total_Stops,
            SUM(CASE WHEN Search_Conducted = TRUE THEN 1 ELSE 0 END) AS Searches,
            SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) AS Arrests
        FROM Police_10
        GROUP BY Violation
    ) AS stats
ORDER BY Search_Rate_Percent DESC, Arrest_Rate_Percent DESC;
"""

#Driver Demographics by Country (Age, Gender, and Race)

query="""
SELECT
    Country_Name,
    Driver_Race,
    COUNT(*) AS Driver_Count,
    ROUND(AVG(Driver_Age), 2) AS Avg_Age
FROM Police_10
GROUP BY Country_Name, Driver_Race
ORDER BY Country_Name, Driver_Count DESC;
"""

#Top 5 Violations with Highest Arrest Rates

query="""
SELECT
    Violation,
    COUNT(*) AS Total_Stops,
    SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) AS Arrests,
    ROUND(SUM(CASE WHEN Is_Arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS Arrest_Rate_Percent
FROM Police_10
GROUP BY Violation
ORDER BY Arrest_Rate_Percent DESC
LIMIT 5;
"""





Mysql connection established successfully
Database created successfully
Table has created successfully in mysql
Data inserted using to_list()
Data inserted Successfully! (No NaN values)
